In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import  train_test_split

In [4]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [3]:
dataPath = "./BostonHousing.csv"
df = pd.read_csv(dataPath)

In [ ]:
df.shape

In [ ]:
x_data = df.iloc[:, :12].values
y_data = df.MEDV.values
print(x_data.shape)
print(y_data.shape)

In [ ]:
# 数据标准化
scaler = StandardScaler()
scaler.fit(x_data)
x_data = scaler.transform(x_data)

In [ ]:
# 数据集划分
X_train, X_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.1, random_state=56
)

In [ ]:
# 转换为张量
train_xt = torch.from_numpy(X_train.astype(np.float32))
train_yt = torch.from_numpy(y_train.astype(np.float32))
test_xt = torch.from_numpy(X_test.astype(np.float32))
test_yt = torch.from_numpy(y_test.astype(np.float32))

In [ ]:
# 数据集加载器
train_data = Data.TensorDataset(train_xt, train_yt)

In [ ]:
train_loader = Data.DataLoader(
    dataset=train_data, batch_size=32, shuffle=True, drop_last=True
)

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_features=12, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=16),
            nn.ReLU(),
            nn.Linear(in_features=16, out_features=1),
        )

    def forward(self, x):
        return self.fc(x)

In [ ]:
model = Model().to(device)

In [ ]:
model

In [ ]:
def train():
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
    loss_fun = nn.MSELoss()
    model.train()
    for epoch in range(100):
        for step, (x, y) in enumerate(train_data):
            out = model(x.to(device))
            loss = loss_fun(torch.squeeze(out.to("cpu")), y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        if epoch % 5 == 0:
            print(f"epoch:{epoch}, loss: {loss.item()}")
    torch.save(model, "./boston.model")

In [ ]:
train()

In [ ]:
@torch.no_grad()
def test(test_xt):
    model = torch.load("./boston.model").to("cpu")
    model.eval()
    out = model(test_xt)
    return out

In [ ]:
predit = test(test_xt)

In [ ]:
predit[6], test_yt[6]